In [2]:
import zipfile, os
import shutil
import tarfile
import re
import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import os
import json
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from pdfrw import PdfReader
import numpy as np
from nltk import ngrams

# elsevier processing

In [82]:
#extraction for elsevier

# update details everytime

#************
RootDir1 = r'/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/NCT_files'

TargetFolder = r'/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/elsevier'


#*****************


import xml.etree.ElementTree as etree
import pandas as pd
bodycount = 0 
full_body_text = []
body_list = []
tagname1 = "{http://purl.org/dc/terms/}title"
tagname2 = "{http://purl.org/dc/terms/}description"
tagname3 = "{http://www.elsevier.com/xml/common/schema}text"
tagname4 = "{http://www.elsevier.com/xml/common/schema}section-title"
tagname5 = "{http://www.elsevier.com/xml/common/schema}para"
tagname6 = "{http://www.elsevier.com/xml/ja/schema}body"
tagname7 = "{http://www.elsevier.com/xml/common/schema}abstract"

tagname8 = "{http://www.elsevier.com/xml/ja/schema}head"
tagname9 = "{http://www.elsevier.com/xml/ja/schema}article"

tagname1 = "{http://purl.org/dc/terms/}title"
tagname6 = "{http://www.elsevier.com/xml/ja/schema}body"
tagname10 = "{http://prismstandard.org/namespaces/basic/2.0/}doi"
tagname11 = "{http://prismstandard.org/namespaces/basic/2.0/}url"

title_list = []
body_list = []
doi_list = []
url_list =[]
abstract_list = []
abstract_fulltext = []
abstractcount = 0
bodycount = 0
titlecount = 0
doicount = 0
urlcount = 0
copied_count = 0
xml_title_list = []
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)  
        xml_name=name
#         print(xml_name)
        with open(SourceFolder) as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'www.elsevier.com' in shorttext:
#             print(SourceFolder)
#                 SourceFolder2 = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)
                copied_count = copied_count + 1
#                 print("copy done for "+ str(name)) 
                
                
                tree = etree.parse(SourceFolder)
                roots = tree.getroot()
                tagname = '{http://www.elsevier.com/xml/ja/schema}body'  
                #get body
                for tag in roots.iter(tagname):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_body_text.append(' '.join(pageText))
                    body_list.append(ElementTree.tostring(tag))
                    bodycount = bodycount + 1
                #get doi    
                for tag in roots.iter(tagname10):
                    doi_list.append(ElementTree.tostring(tag))
                    doicount = doicount + 1

                #get url
                for tag in roots.iter(tagname11):
                    # ElementTree.dump(tag)
                    url_list.append(ElementTree.tostring(tag))
                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    urlcount = urlcount + 1

                #get title
                for tag in roots.iter(tagname1):
                    # ElementTree.dump(tag)
                    title_list.append(ElementTree.tostring(tag))
                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    titlecount = titlecount + 1
                    xml_title_list.append(xml_name)

                #get abstract
                for tag in roots.iter(tagname7):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    abstract_fulltext.append(' '.join(pageText))
                    root_1 = ElementTree.fromstring(tagtext) 
                    abstract_dict = {}  
                    section_title = ""
                    for elem in root_1.iter():
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}title":
                            abstract_dict['Title'] = elem.text
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}section-title":
                            section_title = elem.text
                        if elem.tag == "{http://www.elsevier.com/xml/common/schema}simple-para":
                            abstract_dict[section_title] = elem.text
                    abstract_list.append(abstract_dict)
                    abstractcount = abstractcount + 1



            
#actual title
count = 0
actual_title_list = []
for body in title_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_title_list.append(elem.text)

#actual doi
count = 0
actual_doi_list = []
for body in doi_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_doi_list.append(elem.text)

        
#actual url
count = 0
actual_url_list = []
for body in url_list:
    root = ElementTree.fromstring(body)
    # ElementTree.dump(root)
    for elem in root.iter():
        # print(elem.text)
        count = count + 1
        actual_url_list.append(elem.text)
        
#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "{http://www.elsevier.com/xml/common/schema}section-title"
section_paratag = "{http://www.elsevier.com/xml/common/schema}para"

for body in body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
            # print("_____")
            # print(elem.text)
            # print("_____")
            section_title = elem.text
        if elem.tag == section_paratag:
            # print(elem.text)
            para_dict[section_title] = elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1



import pandas as pd
df_e1 = pd.DataFrame(list(zip(xml_title_list, actual_title_list, actual_body_list, actual_doi_list, actual_url_list,full_body_text,abstract_list,abstract_fulltext)),
               columns =['XML','Title', 'Body', 'DOI','URL','Full_body_text','Abstract','Abstract_full'])
# str(my_dict)
def getFullText(text):
    return str(text)

df_e1['full_text'] = df_e1.Body.apply(getFullText)

df_e2 = pd.concat([df_e1,df_e1['Body'].apply(pd.Series)], axis=1)

df_e3 = pd.concat([df_e2,df_e2['Abstract'].apply(pd.Series)], axis=1)



df_elsiver = df_e3[['XML','Title','Full_body_text','Abstract_full','DOI','URL']]

df_elsiver['Molecule'] = 'Ocrelizumab'
df_elsiver['Publisher_type'] = 'Elsevier'

df_elsiver.shape
# print(copied_count)

df9695aa-09c7-4892-89ae-306766491915.xml
5445f1b8-9ded-478a-a835-076bf5758808.xml
6ccd58f1-8987-46e2-886b-a3b0a8f4b0ab.xml
3dd27480-248e-4e80-a406-c37d91a37014.xml
a9aab8a6-de40-445d-9174-f0f1f9ce00d9.xml
4578e92f-6cdb-43b6-902e-b84c04831180.xml
6d3958f7-d6ba-44bf-9898-95e8ac27d0de.xml
0a55913c-d5af-4081-a640-7a245e6550b3.xml
85ad7cf3-0105-4c33-af31-5625d551f0ab.xml
dc33e4bd-c59b-4527-90c4-23b5a95d4b08.xml
9fcebe47-fc4b-4b33-ae4b-0775fcaf8e69.xml
74b9ef2c-b4e2-417b-bcf3-af38d1919873.xml
5ec62e20-0184-4f2c-be86-c740360d22a0.xml
15416feb-39fa-44f3-9073-27f211b26e6a.xml
3f0fa494-d209-49b4-94f9-39cca7a9e15b.xml
7fe3e2be-7e17-4b7e-bf1f-32c1de626b38.xml
ad028c76-fd8a-4033-b55b-0d47643ef31e.xml
ad5bbd4d-1cde-429b-9806-ef6b778970fd.xml
a388bad2-3f4b-4c9d-a09f-1943df86dfc1.xml
42da067f-5be9-4357-a4b5-00bad7395b1f.xml
f35ef6ea-4a6e-461e-b6c1-972039914957.xml
f3bbe91a-183a-417e-9c28-5dd4bfb35c70.xml
c176b6e4-97f4-43f8-8037-a16b6765cebf.xml
d8b21fad-ea72-4090-b5ca-d1053f582e48.xml
84150de1-d2a7-47

d2f67889-1675-4774-9c9f-c6a8c1f77611.xml
3b4b4084-e7a0-47d2-a52a-655c87ecaded.xml
f5b50c48-ebbd-43b4-8eae-2fcda2da6785.xml
cb86907d-0906-444a-8692-232b3cd26bf2.xml
92cd9796-5849-4f26-826e-81503fa23660.xml
756575b7-8574-45e9-a7c7-b339c6277f9f.xml
1f5ef9dd-d064-4888-babc-f09c721620ea.xml
583cb450-5f5d-4c96-b80e-bdbfb6fee5b6.xml
d032ad86-e1f7-401b-86c7-41c1c51aef66.xml
3cb76eec-a892-4cd7-bd87-1e14a20f7cdc.xml
bf890eb9-6666-4f9d-8c82-8ce8b2f7860a.xml
c12ffadc-e951-4506-a6a3-84cfea64c605.xml
e275850c-039d-4626-8cc0-e4ab8c8ae0f0.xml
7fadf16d-d68b-4568-a4a1-a44745c40ef8.xml
3c6e6c22-e956-4762-98c5-112ef11dff5d.xml
09395a1d-cc30-4059-8403-a852e1396f70.xml
14f99ebf-bd76-48a3-ba54-e62d448702fe.xml
f27befaa-de08-4148-be8d-9089b6d06d5e.xml
13ce727a-1f08-4226-813b-dd2945b2ca43.xml
2b5a370b-ba0f-4dfa-9bd9-b49682532e88.xml
1634f428-46f1-4b5a-879e-8c0f769a48ae.xml
4da1911e-91f3-491b-8015-ebf625418dec.xml
1bd10a42-8aa8-49f4-bd4b-5e59ac175c0c.xml
03989cde-b3cc-430e-bd8d-f4f3e7788196.xml
4257e68f-74ba-4c

c2cf65b3-b9ac-46ad-9b7c-d7d57648a07f.xml
7a9d99a6-91d3-4785-bcc7-1dbc462e3e51.xml
1b338433-72d1-45f5-869e-b057b9c95bfc.xml
edfcce5b-862b-40a9-9538-563018f91190.xml
5304e59e-0eaf-480e-a51a-04139abd9913.xml
95ab8791-23a8-413e-a3a7-34e36532dc25.xml
712dff3d-d1f5-4e46-bb1e-067ffa63aaa7.xml
9fd66f36-b95d-4daa-b9c4-913a5d151ff6.xml
4a3b7684-935e-49c6-a477-890b884705ad.xml
dbffee0c-0f49-486a-9770-a603ffbd0bef.xml
afd9aa74-2e73-4108-bdbb-e5b8052ec546.xml
3001ca62-91a7-4626-bc9d-8346442007c8.xml
f52b8078-ed3a-4851-9326-58675be7abd8.xml
ecfe90f9-25f0-4dce-8e7b-7f125bfc2f46.xml
7dc77b9e-41ff-451f-a52a-ad60d7996fa8.xml
e5924c21-c766-4ef7-8674-33df4bbca899.xml
a5b5663a-1454-429b-bc7e-4bd0e7afffa9.xml
5be5be38-519b-4734-840b-159f80125560.xml
1571134d-b558-4499-9b1f-6039d9667217.xml
cc45a3b4-f16d-48d6-bbf5-c28515dcd050.xml
48a57d6b-20ad-4e4d-b640-d91ab2ac1bfb.xml
50dc55fb-cd66-4090-bf82-b12ac612d49c.xml
d1ff57bd-5591-41d0-9c84-e22e7ce28e84.xml
aebcb514-2ef9-4d10-bf60-dc49ce94fb7c.xml
11a511e3-01de-4f

da32c93e-b332-4e10-b07c-6be7035d575b.xml
ae0e8198-e308-4836-b654-9c58bc1d171c.xml
f70982f9-e651-497e-aa1d-01cf6c181d97.xml
b2371658-8b3b-496a-bbb1-c678af81837b.xml
099b9fa2-e041-491c-b486-1a3d534e2eee.xml
8e8e9d55-7543-488f-80fb-b86cf78cef6c.xml
63f7dfa6-e0ba-4bb0-9074-6a409b562acb.xml
84912cb8-949c-49b2-a8ab-59d74c3e505c.xml
f6331576-9051-45b2-aa23-68b826197125.xml
ba827df2-cb2e-490c-b1da-e484c8a8820f.xml
ed33c249-b022-4474-a7af-8a5072ca7e96.xml
3df5d768-ce33-4e48-9ab9-6c0e14c21da6.xml
2049d1f0-d7e8-4b6c-af84-709b24442c4f.xml
e628add2-da6d-4ba9-99e7-fa225a9851ae.xml
8eea8286-3a70-42ce-be6f-43947ef09c1c.xml
caccb51f-155f-429c-b039-437cf2e30edc.xml
e314d87f-c095-4d9f-a2bd-110ae35d41e5.xml
c4ce84ad-96e7-4282-b0e4-5de11c1a33f3.xml
4d36c250-0f94-4e4b-997b-a20022e07201.xml
0dd20431-e96e-4bed-8fb8-c55054bdf8d3.xml
1ed7a5f7-b2c4-46da-b2e9-a4ff55f98ae2.xml
090c94ee-a10b-49ff-99e7-b57c8d2af05d.xml
c7d70e0b-7b34-464e-b842-f43f605905a1.xml
9e1ad6f1-be42-4fd8-b311-45a0b0b03341.xml
37c5a010-c1cd-4f

ecbb27ea-6179-4b57-b413-47427ab86fa1.xml
f82fb87e-9de7-4541-b07a-1b895ab51757.xml
a757a2a8-5dbe-460f-9083-a6ab185cd27d.xml
20423efc-724b-4b58-805f-17476260e6f3.xml
ef6a0cb9-0220-4e96-a87a-9f1869bc9e82.xml
2b99016c-f64d-44c1-8f2d-f88f4c8a9bb2.xml
df15b190-eafd-4fcc-b2a6-4b3b4fd709fd.xml
2f5b59e1-a241-4906-8883-ec448a16da7d.xml


<ipython-input-82-a794e9b60309>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elsiver['Molecule'] = 'Ocrelizumab'
<ipython-input-82-a794e9b60309>:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elsiver['Publisher_type'] = 'Elsevier'


(227, 8)

In [83]:
APP_FOLDER = '/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/elsevier'

totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
#     print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        if Files.endswith('.xml'):
            totalFiles += 1


print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Total number of files 229
Total Number of directories 0
Total: 229


In [84]:
df_elsiver.head()

XML  \
0  df9695aa-09c7-4892-89ae-306766491915.xml   
1  5445f1b8-9ded-478a-a835-076bf5758808.xml   
2  dc33e4bd-c59b-4527-90c4-23b5a95d4b08.xml   
3  74b9ef2c-b4e2-417b-bcf3-af38d1919873.xml   
4  15416feb-39fa-44f3-9073-27f211b26e6a.xml   

                                               Title  \
0  Emerging therapeutics in the management of con...   
1  The underpinning biology relating to multiple ...   
2  Real-World Data in the Postapproval Setting as...   
3  The role of sex hormones in women with multipl...   
4  Advances in immunotherapeutic targets for chil...   

                                      Full_body_text  \
0  Learning objectives After completing this lear...   
1  1 SARS-Cov-2 and COVID-19 a new pandemic COVID...   
2  Introduction A medical product that has been g...   
3  1 Introduction Multiple sclerosis (MS) is a mu...   
4  1 Introduction Cancer is the second leading ca...   

                                       Abstract_full  \
0  The management of connective tissue diseases i...   
1  Highlights • COVID-19 is a pandemic, sometimes...   
2  Abstract Background SARS-CoV-2 viral infection...   
3  ABSTRACT Purpose This article provides an anal...   
4  Highlights • Gender influences MS prognosis. •...   

                                DOI  \
0        10.1016/j.jaad.2021.12.067   
1       10.1016/j.msard.2020.102174   
2   10.1016/j.clinthera.2021.12.010   
3       10.1016/j.yfrne.2020.100889   
4  10.1016/j.pharmthera.2021.107892   

                                                 URL     Molecule  \
0       http://dx.doi.org/10.1016/j.jaad.2021.12.067  Ocrelizumab   
1      http://dx.doi.org/10.1016/j.msard.2020.102174  Ocrelizumab   
2  http://dx.doi.org/10.1016/j.clinthera.2021.12.010  Ocrelizumab   
3      http://dx.doi.org/10.1016/j.yfrne.2020.100889  Ocrelizumab   
4  http://dx.doi.org/10.1016/j.pharmthera.2021.10...  Ocrelizumab   

  Publisher_type  
0       Elsevier  
1       Elsevier  
2       Elsevier  
3       Elsevier  
4       Elsevier

In [85]:
df_elsiver.shape

(227, 8)

In [86]:
len(set(df_elsiver.URL.to_list()))

206

In [ ]:

df_elsiver.to_excel("/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Processing/Ocrelizumab_elsiver.xlsx")



# WIP

# NLM processing

In [59]:
#extraction for NLM

import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree

RootDir1 = r'/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/NCT_files'

TargetFolder = r'/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/nlm'


#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "title"
section_paratag = "p"
bodycount = 0 
full_body_text = []
body_list = []


nlm_body_list = []
nlm_bodycount = 0
full_body_text_nlm = []
nlm_abstract_list = []
nlm_abstract_count = 0
full_abstract_text = []

tagname1 = 'body' 
tagname2 = 'abstract' 
tagname3 = 'ext-link'
tagname4 = 'ext-link'
tagname5 = 'article-title'
tagname6 = 'front'

file_title = []
abs_dict = {}   
copied_count = 0
# elements = set()
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name)       
        with open(SourceFolder) as name:
            data = name.read()
            text = str(data)
            shorttext = str(data)[:100]
            if 'NLM' in shorttext:
                shutil.copy2(SourceFolder, TargetFolder)
                copied_count = copied_count + 1
#                 print("copy done for "+ str(name)) 
                
                tree = etree.parse(SourceFolder)           
                roots = tree.getroot()  
            
                #get title
                for tag in roots.iter(tagname5):
                    # ElementTree.dump(tag)
                    title_list.append(ElementTree.tostring(tag))

                    # print(".....")
                    # print(tag.text)
                    # print("...")
                    titlecount = titlecount + 1

                title_list = []
                for elem in roots.iter():
    #                 elements.add(elem.tag)
                    if elem.tag == tagname5:
                        title_list.append(elem.text)

                file_title.append(title_list[0])
                
                # get body

                for tag in roots.iter(tagname1):
                    tagtext = ElementTree.tostring(tag)   
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    full_body_text_nlm.append(' '.join(pageText))
                    nlm_body_list.append(ElementTree.tostring(tag))
                    nlm_bodycount = bodycount + 1

                # get abstract

                for tag in roots.iter(tagname2):                    
                    tagtext = ElementTree.tostring(tag)
                    #print(tagtext)
                    tagtext= (str(tagtext, 'UTF-8'))
                    cleanr = re.compile(r'(<(?!/abstract|abstract|p|/p|title|/title).*?>)|(\n)')
                    tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')

                    #print((tagtext))
                    soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                    pageText = soup.findAll(text=True)
                    #print(pageText)
                    root_1 = ElementTree.fromstring(tagtext)

                    for elem in root_1.iter():                   
                        nlm_abstract_list.append(elem.text)
                        nlm_abstract_count = nlm_abstract_count + 1         
                    
                                
     
  
# actual body

for body in nlm_body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    already_exist_title = set()
    count =0
    existing_text = ""
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
#             print("_____")
#             print(elem.text)
#             print("_____")
            section_title = elem.text            
        if elem.tag == section_paratag and section_title not in already_exist_title:
            # print(elem.text)
            para_dict[section_title] = elem.text
            already_exist_title.add(section_title)
        elif elem.tag == section_paratag and section_title in already_exist_title:
#             count = count + 1 
#             print(section_title)
            existing_text = para_dict[section_title]
#             print(existing_text)
            if existing_text:
                if elem.text:
                    para_dict[section_title] = existing_text +  elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1


    
df_nlm_interim = pd.DataFrame(list(zip(file_title,nlm_abstract_list,actual_body_list)),
               columns =['title','abstract','full_text'])


    
df_nlm_interim.head()

title  \
0  The Smartphone App haMSter for Tracking Patien...   
1  Treatment Challenges in Multiple Sclerosis – A...   
2  Clinical trials of disease-modifying agents in...   
3  An update on cladribine for relapsing-remittin...   
4  Impact of methodological choices in comparativ...   

                                            abstract  \
0                                               None   
1                                         Background   
2  Treatment and monitoring decisions in people w...   
3                                          Objective   
4  We aim to provide a rationale and pilot study ...   

                                           full_text  
0  {'Multiple Sclerosis and Patient-Reported Outc...  
1  {'Introduction': 'Glatiramer acetate (GA) was ...  
2  {'': 'The diagnosis of multiple sclerosis (MS)...  
3  {'Introduction': 'The number of people living ...  
4  {'Background': 'Natalizumab [In the absence of...

# WIP

# Wiley

In [43]:
#extraction of wiley

import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree

RootDir1 = r'/Users/savioel2/Library/CloudStorage/OneDrive-NovartisPharmaAG/HAWK AI 2022/Insights Module/NKC_DH_Data_and_Code/NKC DH Data zips/Ocrelizumab/NCT_files'


#actual body
count = 0
actual_body_list = []
elements = ""
section_titletag = "{http://www.wiley.com/namespaces/wiley}title"
section_paratag = "{http://www.wiley.com/namespaces/wiley}p"
bodycount = 0 
full_body_text = []
body_list = []

doi_list = []
wiley_body_list = []
wiley_bodycount = 0
full_body_text_wiley = []
file_title = []
file_doi = []
abstract_list = []
count = 0
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
    for name in files:
        SourceFolder = os.path.join(root,name) 
#         print(SourceFolder)
        try:
            with open(SourceFolder) as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:200]
                if 'www.wiley.com' in shorttext:
                    tree = etree.parse(SourceFolder)  
#                     print(SourceFolder)
    #                 count = count +1 
                    roots = tree.getroot()
                    tagname = '{http://www.wiley.com/namespaces/wiley}body' 
                    for tag in roots.iter(tagname):
                        tagtext = ElementTree.tostring(tag)   
                        soup = BeautifulSoup(tagtext,features="lxml")    # txt is simply the a string with your XML file
                        pageText = soup.findAll(text=True)
                        full_body_text_wiley.append(' '.join(pageText))
                        wiley_body_list.append(ElementTree.tostring(tag))
                        wiley_bodycount = wiley_bodycount + 1


                     #get title

                    title_list = []
                    tagname5 = '{http://www.wiley.com/namespaces/wiley}articleTitle'
                    for elem in roots.iter():
        #                 elements.add(elem.tag)
                        if elem.tag == tagname5:
                            title_list.append(elem.text)

                    file_title.append(title_list[0])
                    
                    tagname6 = '{http://www.wiley.com/namespaces/wiley}doi'
                    for elem in roots.iter():
        #                 elements.add(elem.tag)
                        if elem.tag == tagname6:
                            doi_list.append(elem.text)
#                             print(elem.text)
                            
                    file_doi.append(doi_list[1])
        
                    #get abstract 
            
                    tagname7 = '{http://www.wiley.com/namespaces/wiley}abstract'
                    for elem in roots.iter():
        #                 elements.add(elem.tag)
                        if elem.tag == tagname7:
                            abstract_list.append(elem.text)
#                             print(elem.text)
                            
#                     file_doi.append(doi_list[1])

        except:
            check = 2
            
        

# print(count)
for body in wiley_body_list:
    root = ElementTree.fromstring(body)    
    # ElementTree.dump(root)
    elements = ""
    section_title = ""
    para_dict = {}
    already_exist_title = set()
    count =0
    existing_text = ""
    for elem in root.iter():
        # print(elem.tag)
        if elem.tag == section_titletag:
#             print("_____")
#             print(elem.text)
#             print("_____")
            section_title = elem.text            
        if elem.tag == section_paratag and section_title not in already_exist_title:
            # print(elem.text)
            para_dict[section_title] = elem.text
            already_exist_title.add(section_title)
        elif elem.tag == section_paratag and section_title in already_exist_title:
#             count = count + 1 
#             print(section_title)
            existing_text = para_dict[section_title]
#             print(existing_text)
            if existing_text:
                if elem.text:
                    para_dict[section_title] = existing_text +  elem.text
        try:    
            elements = elements + elem.text
        except:
            elements = elements + ""
    actual_body_list.append(para_dict)
    count = count + 1

# #actual doi
# count = 0
# actual_doi_list = []
# for body in doi_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_doi_list.append('None')

        
# #actual url
# count = 0
# actual_url_list = []
# for body in url_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_url_list.append('None')
    
# #actual abstract
# count = 0
# actual_abstract_list = []
# for body in url_list:
# #     root = ElementTree.fromstring(body)
# #     # ElementTree.dump(root)
# #     for elem in root.iter():
# #         # print(elem.text)
# #         count = count + 1
#         actual_abstract_list.append('None')


df_wiley = pd.DataFrame(list(zip(file_title, actual_body_list, full_body_text_wiley,abstract_list,doi_list)),
               columns =['Title','Body','Full_body_text','abstract','doi'])
# str(my_dict)
def getFullText(text):
    return str(text)

df_wiley['full_text'] = df_wiley.Body.apply(getFullText)

df_wiley = pd.concat([df_wiley,df_wiley['Body'].apply(pd.Series)], axis=1)

df_wiley = df_wiley[['Title','Full_body_text','abstract','doi']]
df_wiley.shape

(94, 4)

In [44]:
df_wiley.head()

Title  \
0  Alpha lipoic acid inhibits T cell migration in...   
1  Distinction between MOG antibody‐positive and ...   
2  The pathogenesis and diagnosis of systemic lup...   
3                    Lupus nephritis: current update   
4                                 Multiple sclerosis   

                                      Full_body_text abstract  \
0  \n \n Multiple sclerosis (MS) is the most comm...       \n   
1  \n \n Myelin oligodendrocyte glycoprotein (MOG...       \n   
2  \n \n Systemic lupus erythematosus (SLE) is a ...       \n   
3  \n \n Approximately 60% of patients with syste...       \n   
4  \n \n Multiple sclerosis (MS) is the most comm...       \n   

                       doi  
0  10.1002/(ISSN)1552-4604  
1       10.1002/jcph.v60.8  
2        10.1002/jcph.1605  
3  10.1002/(ISSN)1531-8249  
4        10.1002/ana.v89.1